## Regression - ResNet34 


In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# setup CUDA_VISIBLE DEVICES for titan.sci.utah.edu
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"


In [ ]:
# This file contains all the main external libs we'll use

from fastai.imports import *
from fastai.conv_learner import *

from fastai.plots import *


PATH = "../data/"
sz=224
arch=resnet34
bs=8

In [ ]:
label_csv = f'{PATH}Dataset.csv'
n = len(list(open(label_csv)))-1
# Return validation indexes using a 10% split
val_idxs = get_cv_idxs(n,val_pct=0.1)
print('n:',n)
print('Nb val_idxs',len(val_idxs))
print('val_idxs',val_idxs)

In [ ]:
label_df = pd.read_csv(label_csv)
label_df.head()

In [ ]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on)
data = ImageClassifierData.from_csv(PATH, 'data_all', label_csv, tfms=tfms, continuous = True, bs=bs, val_idxs=val_idxs, num_workers=10)

# Add simple dense model (instead of default fastai model)
#head_reg = nn.Sequential(Flatten(), nn.Linear(25088,1))
learn = ConvLearner.pretrained(arch, data, xtra_fc=[], ps=0.25, precompute=False)
# Add dropout
#learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)

In [ ]:
learn.opt_fn = optim.Adam
learn.crit = nn.L1Loss()

In [ ]:
learn

In [ ]:
lrf=learn.lr_find()

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.sched.plot()

In [ ]:
#learn.fit(5e-3, 5)
#learn.precompute=False

In [ ]:
lr = 2e-1

In [ ]:
learn.fit(lr, 5)

In [ ]:
learn.fit(lr, 5, cycle_len=1)

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.save('224_lastlayer_ResNet34_Regression')

In [ ]:
learn.load('224_lastlayer_ResNet34_Regression')

In [ ]:
learn.unfreeze()
lr = 1e-2
lrs=np.array([lr/9,lr/3,lr])

In [ ]:
lrf=learn.lr_find()

In [ ]:
learn.sched.plot()

In [ ]:
lr = 5e-4
lrs=np.array([lr/9,lr/3,lr])
learn.fit(lr, 5, cycle_len = 1, cycle_mult = 2)

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.save('224_all_ResNet34_Regression')

In [ ]:
learn.load('224_all_ResNet34_Regression')

In [ ]:
preds, y = learn.TTA()
#print(preds)
print(y)
#log_preds,y = learn.predict()
probs = np.mean(preds,0)
print(probs)
#accuracy_np(probs,y)

## Analyzing results

In [ ]:
from sklearn.metrics import *
import math
MAE = mean_absolute_error(y, probs)
RMSE = math.sqrt(mean_squared_error(y, probs))
print('MAE', MAE)
print('RMSE', RMSE)

In [ ]:
learn.summary()